In [ ]:
import re
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data processing

In [ ]:
root = "./"
prog = re.compile("^pfs.+txt$")
output_list = [f for f in os.listdir(root) if prog.match(f)]

for fname in output_list:
    df     = pd.read_table(fname, header=None, sep='\t').replace(-1, np.nan)
    df.columns = ["resM", "bK", "cr", "status", "rd", "pSize", "cProp", "cDeg", "lastna", "IC"]

    rd     = df.rd.max()+1
    lv     = int(df.resM.max())
    df_new = pd.DataFrame(np.zeros([lv, 11]), columns = ["resM", "bK", "cr", "status", "pSizeM", "pSizeSD", "cPropM", "cPropSD", 
                                                        "cDegM", "cDegSD", "IC"])

    for i in range(lv):
        tmp = df[df.resM == i+1].reset_index(drop=True)
        df_new.loc[i, ["resM", "bK", "cr", "status", "IC"]] = list(tmp.loc[0, ["resM", "bK", "cr", "status", "IC"]])
        df_new.loc[i, ["pSizeM", "cPropM", "cDegM"]]        = list(tmp.loc[:, ["pSize", "cProp", "cDeg"]].mean())
        df_new.loc[i, ["pSizeSD", "cPropSD", "cDegSD"]]     = list(tmp.loc[:, ["pSize", "cProp", "cDeg"]].std())

    df_new.to_csv(fname[:-4]+".csv", index=False)

In [ ]:
def concat_all_df(f_list, out_fname):
    for i, v in enumerate(f_list):
        df = pd.read_csv(v)
        if i == 0:
            df_new = df
        else:
            df_new = pd.concat([df_new, df], axis = 0, ignore_index=True)
    df_new.to_csv(out_fname, index=False)

bK_arr = np.arange(1.0, 6.6, 0.25)
f_list = [f"pfs_social_bK{int(np.ceil(bK*10))}.csv" for bK in np.arange(1.0, 6.6, 0.25)]
concat_all_df(f_list, "pfs_social.csv")

## Visualization

In [ ]:
df = pd.read_csv("pfs_social.csv")

plt.figure(figsize = (4, 3))
table = df.pivot("bK", "resM", "cPropM")
ax1 = sns.heatmap(table, square = True, cmap = "Wistia", cbar_kws={'label': 'Proportion of cooperators'})
ax1.invert_yaxis()
plt.xlabel("Mean resource availability")
plt.ylabel("Cooperation efficiency")
plt.savefig("figure_2.pdf", format="pdf", dpi=300)

In [ ]:
def my_errorbar(df_social, df_nonsocial, mode):
    if mode == 1:
        plt.ylim(-0.15, 1.7)
        plt.errorbar(df_social.resM, df_social.cPropM, yerr=df_social.cPropSD, fmt='o', color = "#e48036", elinewidth=2, capsize=3, markersize=4, label='Social population')
        plt.errorbar(df_nonsocial.resM, df_nonsocial.cPropM, yerr=df_nonsocial.cPropSD, fmt='^', color = "#6c78b8", elinewidth=2, capsize=3, markersize=4, label='Nonsocial population')
        plt.xlabel("Environmental resource availability", labelpad=8)
        plt.ylabel("Proportion of cooperators", labelpad=8)
        plt.legend(loc = 1, frameon = False, bbox_to_anchor=(1.05, 1))
    elif mode == 2:
        plt.ylim(-0.05, 1.0)
        plt.errorbar(df_social.resM, df_social.cDegM, yerr=df_social.cDegSD, fmt='o', color = "#e48036", elinewidth=2, capsize=3, markersize=4, label='Social population')
        plt.errorbar(df_nonsocial.resM, df_nonsocial.cDegM, yerr=df_nonsocial.cDegSD, fmt='^', color = "#6c78b8", elinewidth=2, capsize=3, markersize=4, label='Nonsocial population')
        plt.xlabel("Environmental resource availability", labelpad=8)
        plt.ylabel("Average degree of cooperation", labelpad=8)
        plt.legend(loc = 1, frameon = False, bbox_to_anchor=(1.05, 1))
    elif mode == 3:
        plt.ylim(-100, 2000)
        plt.errorbar(df_social.resM, df_social.pSizeM, yerr=df_social.pSizeSD, fmt='o', color = "#e48036", elinewidth=2, capsize=3, markersize=4, label='Social population')
        plt.errorbar(df_nonsocial.resM, df_nonsocial.pSizeM, yerr=df_nonsocial.pSizeSD, fmt='^', color = "#6c78b8", elinewidth=2, capsize=3, markersize=4, label='Nonsocial population')
        plt.xlabel("Environmental resource availability", labelpad=8)
        plt.ylabel("Population size", labelpad=8)
        plt.legend(loc = 1, frameon = False, bbox_to_anchor=(1.05, 1))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.figure(figsize=(10, 6.67))
plt.subplots_adjust(wspace=0.6, hspace=0.7)
df_social    = pd.read_csv("pfs_social_bK10.csv")
df_nonsocial = pd.read_csv("pfs_nonsocial_bK10.csv")
plt.subplot(231)
my_errorbar(df_social, df_nonsocial, 1)
plt.subplot(234)
my_errorbar(df_social, df_nonsocial, 2)
df_social    = pd.read_csv("pfs_social_bK30.csv")
df_nonsocial = pd.read_csv("pfs_nonsocial_bK30.csv")
plt.subplot(232)
plt.ylim(-0.15, 1.7)
my_errorbar(df_social, df_nonsocial, 1)
plt.subplot(235)
plt.ylim(-0.05, 1.0)
my_errorbar(df_social, df_nonsocial, 2)
df_social    = pd.read_csv("pfs_social_bK50.csv")
df_nonsocial = pd.read_csv("pfs_nonsocial_bK50.csv")
plt.subplot(233)
my_errorbar(df_social, df_nonsocial, 1)
plt.subplot(236)
my_errorbar(df_social, df_nonsocial, 2)
plt.savefig("figure_3.pdf", format="pdf", dpi=300)

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.figure(figsize=(10, 2.8))
plt.subplots_adjust(wspace=0.6, hspace=0.7)
df_social    = pd.read_csv("pfs_social_bK10.csv")
df_nonsocial = pd.read_csv("pfs_nonsocial_bK10.csv")
plt.subplot(131)
my_errorbar(df_social, df_nonsocial, 3)
df_social    = pd.read_csv("pfs_social_bK30.csv")
df_nonsocial = pd.read_csv("pfs_nonsocial_bK30.csv")
plt.subplot(132)
my_errorbar(df_social, df_nonsocial, 3)
df_social    = pd.read_csv("pfs_social_bK50.csv")
df_nonsocial = pd.read_csv("pfs_nonsocial_bK50.csv")
plt.subplot(133)
my_errorbar(df_social, df_nonsocial, 3)
plt.savefig("figure_5.pdf", format="pdf", dpi=300)